In [1]:
from sklearn import metrics
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np
from sklearn import ensemble
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pandas as pd
import scipy
from scipy.stats import chi2
import matplotlib.pyplot as plt
from sklearn.utils.multiclass import type_of_target
from scipy import stats
import xgboost as xgb
from sklearn.metrics import auc,roc_curve,roc_auc_score
from sklearn.externals import joblib
import graphviz
%matplotlib inline 

ModuleNotFoundError: No module named 'xgboost'

In [3]:
import os
os.chdir(r'E:\AnalystPersonal\chenxiaoyan\催收\催收929\xgboost')

In [4]:
#分数转化
def score_ys(datain,p,PDO,basepoint):
    cs=np.log(p/(1-p))
    B=PDO/np.log(2)
    A=basepoint+B*np.log(1/50)
    score=np.round(A-cs*B)
    datain['score'] = score
    datain['p']=p

In [5]:
def pct_rank_qcut(datain,series,n):
   
    """离散型变量分箱"""
    edages = pd.Series([i/n for i in range(n)]) #生成分位数
    
    func = lambda x:(edages >= x).idxmax() #argmax()返回最大值对应的索引,索引从0开始计算； func函数：输入x输出，输出百分位数对应的索引值
    datain[series + '_bin' ] = datain[series].rank(na_option = 'top',pct=True).astype(float).apply(func) 

In [6]:
oot = pd.read_csv('oot_10.csv',na_values =['',' ','.','-1','-2'],engine='python')

In [19]:
#oot['zl'].astype('int64')

In [7]:
#分账龄建模 - m1-m2 - 使用相同变量
oot_m12 = oot[oot['zl'].isin([1,2])][['default','zl','last_pay_days','self_FR_get_SR_get_prop_6m','self_FR_get_SR_get_prop_tt',
            'recent_first_days','fp_camt_pct','o_interest','o_total_amount','return_amt']]
print(oot_m12.shape)
print(oot_m12['default'].value_counts(normalize=True))

(42451, 10)
0    0.827001
1    0.172999
Name: default, dtype: float64


In [8]:
booster= joblib.load(r'm12.m')

In [9]:
xgb_oot = xgb.DMatrix(oot_m12[oot_m12.drop(['default','zl','o_total_amount','return_amt'],axis=1).columns],label=oot_m12.default)

In [10]:
def report_performance(preds, y,cutoff = 0.5):
    ks =  stats.ks_2samp(preds[y==1], preds[y!=1]).statistic
    gini =  metrics.roc_auc_score(y,preds)*2 - 1.0
    accuracy = metrics.accuracy_score(y, np.where(preds > cutoff,1,0))
    recall = metrics.recall_score(y, np.where(preds > cutoff,1,0))
    precision = metrics.precision_score(y, np.where(preds > cutoff,1,0))
    cm = metrics.confusion_matrix(y, np.where(preds > cutoff,1,0)) 
    # fpr, tpr, thresholds = metrics.roc_curve(y,preds) 
    auc = metrics.roc_auc_score(y,preds)
    # roc_auc = metrics.auc(fpr,tpr)
    print('=======================report=================')
    print('Gini is: %-.5f \t KS is: %-.5f \t accuracy: %-.5f \t Precision: %-.5f \t Recall:%-.5f \t AUC:%-.5f' % (gini, ks, accuracy,precision,recall,auc))
    print('confusion matrix:')
    print(cm)

In [11]:
preds=booster.predict(xgb_oot,ntree_limit = booster.best_ntree_limit)
report_performance(preds, oot_m12.default)

=======================report=================
Gini is: 0.29900 	 KS is: 0.22080 	 accuracy: 0.82700 	 Precision: 0.00000 	 Recall:0.00000 	 AUC:0.64950
confusion matrix:
[[35107     0]
 [ 7344     0]]


D:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
preds

array([0.29192474, 0.2913384 , 0.29103908, ..., 0.29192474, 0.29192474,
       0.29192474], dtype=float32)

In [48]:
score_ys(oot_m12,preds,50,500)

### 计算z值

## M3-M6

In [13]:
booster= joblib.load(r'm36.m')

In [14]:
oot_m36 = oot[oot['zl'].isin ([3,4,5,6])][['default','zl','last_pay_days','self_FR_get_SR_get_prop_6m','self_FR_get_SR_get_prop_tt',
            'recent_first_days','fp_camt_pct','o_interest','o_total_amount','return_amt']]

In [15]:
print(oot_m36.shape)
print(oot_m36['default'].value_counts(normalize=True))

(69991, 10)
0    0.972596
1    0.027404
Name: default, dtype: float64


In [16]:
booster= joblib.load(r'm36.m')

In [17]:
xgb_oot = xgb.DMatrix(oot_m36[oot_m36.drop(['default','zl','o_total_amount','return_amt'],axis=1).columns],label=oot_m36.default)

In [18]:
preds=booster.predict(xgb_oot,ntree_limit = booster.best_ntree_limit)
report_performance(preds, oot_m36.default)

=======================report=================
Gini is: 0.50719 	 KS is: 0.37537 	 accuracy: 0.97260 	 Precision: 0.00000 	 Recall:0.00000 	 AUC:0.75359
confusion matrix:
[[68073     0]
 [ 1918     0]]


D:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [19]:
preds

array([0.03257057, 0.03257057, 0.03257057, ..., 0.02814481, 0.03594322,
       0.03325623], dtype=float32)

In [57]:
score_ys(oot_m36,preds,50,500)

## M7+

In [20]:
booster= joblib.load(r'm7.m')

In [21]:
oot_m7 = oot[oot['zl']>6][['default','zl','last_pay_days','self_FR_get_SR_get_prop_6m','self_FR_get_SR_get_prop_tt',
            'recent_first_days','fp_camt_pct','o_interest','o_total_amount','return_amt']]

In [22]:
print(oot_m7.shape)
print(oot_m7['default'].value_counts(normalize=True))

(750230, 10)
0    0.994363
1    0.005637
Name: default, dtype: float64


In [23]:
xgb_oot = xgb.DMatrix(oot_m7[oot_m7.drop(['default','zl','o_total_amount','return_amt'],axis=1).columns],label=oot_m7.default)

In [24]:
preds=booster.predict(xgb_oot,ntree_limit = booster.best_ntree_limit)
report_performance(preds, oot_m7.default)

D:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


=======================report=================
Gini is: 0.63567 	 KS is: 0.50489 	 accuracy: 0.99436 	 Precision: 0.00000 	 Recall:0.00000 	 AUC:0.81783
confusion matrix:
[[746001      0]
 [  4229      0]]


In [25]:
preds

array([0.00184179, 0.00184179, 0.00184179, ..., 0.00138605, 0.00844154,
       0.00144677], dtype=float32)

In [64]:
score_ys(oot_m7,preds,50,500)

In [65]:
oot_ = pd.concat([oot_m12,oot_m36,oot_m7])

In [66]:
oot_

,default,zl,last_pay_days,self_FR_get_SR_get_prop_6m,self_FR_get_SR_get_prop_tt,recent_first_days,fp_camt_pct,o_interest,o_total_amount,return_amt,score,p
1456,0,2,1020,0.000000,0.000000,0.0,0.350411,0.00,21076.25,0.0,282.0,0.291925
19021,0,1,371,0.000000,0.000000,0.0,2.037588,1159.36,2639.72,0.0,282.0,0.291338
19027,0,2,726,0.000000,0.000000,0.0,12.504921,977.03,11393.34,0.0,282.0,0.291039
19072,0,2,665,0.000000,0.000000,0.0,0.271889,241.57,3891.44,0.0,284.0,0.284378
19080,0,2,645,1.000000,1.000000,0.0,3.290934,879.64,13699.53,0.0,282.0,0.289708
19098,0,2,634,1.000000,1.000000,0.0,0.186118,306.01,6950.45,0.0,285.0,0.283426
19109,0,1,624,0.000000,1.000000,0.0,0.079726,54.69,2853.04,0.0,283.0,0.288984
19110,0,1,624,0.000000,1.000000,0.0,0.139396,0.00,22683.44,0.0,284.0,0.285212
19165,0,2,473,1.000000,1.000000,0.0,4.947673,828.26,4528.65,0.0,282.0,0.289708
19192,0,1,371,0.000000,1.000000,0.0,2.090141,718.12,2363.97,0.0,282.0,0.289708


In [67]:
pct_rank_qcut(oot_,'score',20)
pct_rank_qcut(oot_,'last_pay_days',10)
pct_rank_qcut(oot_,'recent_first_days',10)
pct_rank_qcut(oot_,'self_FR_get_SR_get_prop_6m',10)
pct_rank_qcut(oot_,'fp_camt_pct',10)
pct_rank_qcut(oot_,'o_interest',10)
pct_rank_qcut(oot_,'self_FR_get_SR_get_prop_tt',10)

In [70]:
oot_.to_csv('oot_10_c.csv')

In [68]:
report_performance(oot_.p, oot_.default)

D:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


=======================report=================
Gini is: 0.80662 	 KS is: 0.68850 	 accuracy: 0.98436 	 Precision: 0.00000 	 Recall:0.00000 	 AUC:0.90331
confusion matrix:
[[849181      0]
 [ 13491      0]]


In [71]:
print(oot_['default'].value_counts(normalize=True))

0    0.984361
1    0.015639
Name: default, dtype: float64
